## filtered csv common_titles

it creates 2 dictionaries:
* normalized titles and the number of appearences
* normalized titles and its 1000 most common tracks

In [1]:
import pandas as pd
import numpy as np
import sys
import json
import re
import collections
import os## filtered csv common_titles
import datetime
import csv
from gensim.models import Word2Vec
from tqdm import tqdm_notebook
from whoosh.analysis import CharsetFilter, StemmingAnalyzer
from whoosh import fields
from whoosh.support.charset import accent_map
import json
import re
import pandas as pd
import numpy as np
from whoosh.analysis import CharsetFilter, StemmingAnalyzer
from whoosh import fields
from whoosh.support.charset import accent_map
from nltk.stem import PorterStemmer
from string import ascii_letters
from utils import normalize_name


quick = False
max_files_for_quick_processing = 1

mpd_path = '../../MPD/data/' #DEFINE YOUR PATH

In [ ]:
total_playlists = 0
title_histogram = collections.Counter()
dict_sorted_titles = dict()

def process_mpd(path):
    print("making dictionary of titles...")
    create_common_titles(path)
    reorder()
    print("filling dictionary of titles...")
    fill_common_titles(path)
    print("cuting to best 1000 tracks...")
    cut_1000()   
    np.save('dictionaries/ntitles_1000tracks_final', dict_sorted_titles) 
    print("dictionaries successfully saved")
    show_summary()

def create_common_titles(path):
    global total_playlists
    count = 0
    filenames = os.listdir(path)
    for filename in tqdm_notebook(sorted(filenames)):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice['playlists']:
                process_playlist(playlist)
                total_playlists+=1
            count += 1

            if quick and count > max_files_for_quick_processing:
                break
                
def reorder():
    global dict_sorted_titles
    if not os.path.exists("dictionaries"):
        os.makedirs("dictionaries")
    dict_sorted_titles = collections.OrderedDict(title_histogram.most_common())
    np.save('dictionaries/dict_ntitles_final', dict_sorted_titles)
    for word in dict_sorted_titles:
        dict_sorted_titles[word]=collections.Counter()
    
        
        
def fill_common_titles(path):
    global dict_sorted_titles
    count = 0
    filenames = os.listdir(path)
    for filename in tqdm_notebook(sorted(filenames)):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice['playlists']:
                title = normalize_name(playlist["name"])
                if title in dict_sorted_titles.keys():
                    for track in playlist["tracks"]:
                        dict_sorted_titles[title][track["track_uri"]] += 1    
                else: print(title)
                             
            count += 1

            if quick and count > max_files_for_quick_processing:
                break
                
def cut_1000():
    
     # cut to 1000 best tracks 
    for title in dict_sorted_titles:
        if type(dict_sorted_titles[title])!=collections.Counter:
            print(title,type(dict_sorted_titles[title]))
        else:
            dict_sorted_titles[title] = dict_sorted_titles[title].most_common(1000)
            #genre_tracks[ti] = [l[0] for l in genre_tracks[ti]]
            #if len(genre_tracks[ti])<600:
            #    not_enough+=1 
    

def show_summary():
    print ("number of playlists", total_playlists)
    print ("number of unique normalized titles", len(dict_sorted_titles))
        


def process_playlist(playlist):
    nname = normalize_name(playlist['name'])
    title_histogram[nname] += 1


process_mpd(mpd_path)

making dictionary of titles...



filling dictionary of titles...



cuting to best 1000 tracks...
dictionaries successfully saved
number of playlists 1000000
number of unique normalized titles 15875


In [ ]:
total_playlists = 0
artist_histogram = collections.Counter()
dict_sorted_artists = dict()

quick = False
max_files_for_quick_processing = 1

def process_mpd(path):
    print("making dictionary of artists...")
    create_common_artists(path)
    reorder()
    print("filling dictionary of artists...")
    fill_common_artists(path)
    print("cuting to best 1000 tracks...")
    cut_1000()   
    np.save('dictionaries/nartists_1000tracks_final', dict_sorted_artists) 
    print("dictionaries successfully saved")
    show_summary()

def create_common_artists(path):
    global total_playlists
    count = 0
    filenames = os.listdir(path)
    for filename in tqdm_notebook(sorted(filenames)):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice['playlists']:
                process_playlist(playlist)
                total_playlists+=1
            count += 1

            if quick and count > max_files_for_quick_processing:
                break
                
def reorder():
    global dict_sorted_artists
    dict_sorted_artists = collections.OrderedDict(artist_histogram.most_common())
    np.save('dictionaries/dict_nartists_final', dict_sorted_artists)
    for word in dict_sorted_artists:
        dict_sorted_artists[word]=collections.Counter()
    
    print(artist_histogram.most_common(100))
    
        
        
def fill_common_artists(path):
    global dict_sorted_artists
    count = 0
    filenames = os.listdir(path)
    for filename in tqdm_notebook(sorted(filenames)):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice['playlists']:
                for track in playlist["tracks"]:
                    artist = normalize_name(track["artist_name"])
                    if artist in dict_sorted_artists.keys():
                        dict_sorted_artists[artist][track["track_uri"]] += 1    
                    else: print(artist)

            count += 1

            if quick and count > max_files_for_quick_processing:
                break
                
def cut_1000():
    
     # cut to 1000 best tracks 
    for artist in dict_sorted_artist:
        if type(dict_sorted_artists[artist])!=Counter:
            print(artist,type(dict_sorted_artists[artist]))
        else:
            dict_sorted_artists[artist] = dict_sorted_artists[artist].most_common(1000)
            #genre_tracks[ti] = [l[0] for l in genre_tracks[ti]]
            #if len(genre_tracks[ti])<600:
            #    not_enough+=1 
    

def show_summary():
    print ("number of unique normalized artists", len(dict_sorted_artists))
        


def process_playlist(playlist):
    for track in playlist["tracks"]:
        nname = normalize_name(track['artist_name'])
        artist_histogram[nname] += 1


process_mpd(mpd_path)

making dictionary of artists...



[('drake', 847163), ('kanye west', 413297), ('kendrick lamar', 353624), ('rihanna', 339570), ('weeknd', 316608), ('eminem', 294667), ('ed sheeran', 272116), ('futur', 250854), ('justin bieber', 243119), ('cole', 241572), ('beyonce', 230857), ('chainsmok', 223509), ('chri brown', 212772), ('calvin harri', 203047), ('twenti on pilot', 198905), ('lil uzi vert', 197855), ('post malon', 195907), ('jai', 193999), ('eazi', 193708), ('big sean', 192478), ('maroon', 187033), ('migo', 184458), ('coldplai', 182350), ('lil wayne', 176657), ('rae sremmurd', 168351), ('imagin dragon', 167940), ('bruno mar', 162464), ('childish gambino', 159872), ('nicki minaj', 152655), ('luke bryan', 146260), ('wiz khalifa', 145689), ('fall out boi', 142767), ('ariana grand', 142115), ('usher', 141505), ('logic', 140449), ('john mayer', 139947), ('travi scot', 139928), ('on direct', 139554), ('chanc rapper', 137960), ('dj khale', 137523), ('panic disco', 136328), ('florida georgia line', 135666), ('red hot chili p